In [47]:
!pip install requests

import requests
from urllib.parse import urljoin
s = requests.Session()
# Save credentials into session
s.auth = ("username", "password")

# Send a POST request to the /authentication API
response = s.post('https://api.worldquantbrain.com/authentication')
if response.status_code == requests.status_codes.codes.unauthorized:
    if response.headers["WWW-Authenticate"] == "persona":
        # Outputs the URL to access through the browser to complete biometrics authentication
        input("Complete biometrics authentication and press any key to continue: " + urljoin(response.url, response.headers["Location"]))
        response = s.post(urljoin(response.url, response.headers["Location"]))
    else:
        print("incorrect email and password")

In [48]:
!pip install -r requirements.txt

In [3]:
import brain_lib as brain
import helpful_functions as hf
import pandas as pd
import requests
import plotly.express as px

In [4]:
s = brain.start_session()


Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_rXrUp8PeqzbpaG9EG79Zurah




## IMPORTING DATASETS AND SELECTING DATAFIELDS

### In line with our hypothesis indicating the significant influence of implied volatility on option pricing, we selected 'option8' .


In [5]:
dataset_id = 'option8'
dataset_id

'option8'

In [6]:
# Loading all datafields in 'option8' data set
datafields_df = hf.get_datafields(s, dataset_id=dataset_id)
datafields_df

,id,description,dataset,category,subcategory,region,delay,universe,type,coverage,turnover,userCount,alphaCount
0,historical_volatility_10,Close-to-close Historical volatility over 10 days,"{'id': 'option8', 'name': 'Volatility Data'}",option,option-option-volatility,USA,1,TOP3000,MATRIX,0.6983,None,235,565
1,historical_volatility_120,Close-to-close Historical volatility over 120 ...,"{'id': 'option8', 'name': 'Volatility Data'}",option,option-option-volatility,USA,1,TOP3000,MATRIX,0.7004,None,177,670
2,historical_volatility_150,Close-to-close Historical volatility over 150 ...,"{'id': 'option8', 'name': 'Volatility Data'}",option,option-option-volatility,USA,1,TOP3000,MATRIX,0.7014,None,105,319
3,historical_volatility_180,Close-to-close Historical volatility over 180 ...,"{'id': 'option8', 'name': 'Volatility Data'}",option,option-option-volatility,USA,1,TOP3000,MATRIX,0.7023,None,202,14401
4,historical_volatility_20,Close-to-close Historical volatility over 20 days,"{'id': 'option8', 'name': 'Volatility Data'}",option,option-option-volatility,USA,1,TOP3000,MATRIX,0.6983,None,142,481
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,parkinson_volatility_180,Parkinson models' Historical volatility over 1...,"{'id': 'option8', 'name': 'Volatility Data'}",option,option-option-volatility,USA,1,TOP3000,MATRIX,0.7022,None,145,459
60,parkinson_volatility_20,Parkinson models' Historical volatility over 2...,"{'id': 'option8', 'name': 'Volatility Data'}",option,option-option-volatility,USA,1,TOP3000,MATRIX,0.6983,None,85,250
61,parkinson_volatility_30,Parkinson models' Historical volatility over 3...,"{'id': 'option8', 'name': 'Volatility Data'}",option,option-option-volatility,USA,1,TOP3000,MATRIX,0.6981,None,76,177
62,parkinson_volatility_60,Parkinson models' Historical volatility over 6...,"{'id': 'option8', 'name': 'Volatility Data'}",option,option-option-volatility,USA,1,TOP3000,MATRIX,0.6984,None,82,398


In [7]:
# Simulating all datafields of the whole dataset 
expression_list =[f'{x}' for x  in datafields_df.id.values.tolist()]

In [8]:
alpha_list = [brain.generate_alpha(x, region= "USA",truncation = 0.08, universe = "TOP3000",pasteurization = 'ON', decay = 4 ,neutralization = "SUBINDUSTRY") for x in expression_list]

In [9]:
result = brain.simulate_alpha_list_multi(s, alpha_list)

100%|███████████████████████████████████████████| 22/22 [10:47<00:00, 29.44s/it]


In [10]:
result_st1 = hf.prettify_result(result, detailed_tests_view=False)
result_st1

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,6633489,20000000,1059,1410,0.1442,0.0674,0.4422,0.000936,0.34,0.50,On6vdNq,implied_volatility_call_180,FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
1,6537068,20000000,1060,1409,0.1479,0.0665,0.4457,0.000899,0.34,0.50,8OGqWjz,implied_volatility_call_270,FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
2,6532091,20000000,1039,1411,0.1311,0.0664,0.5155,0.001013,0.33,0.46,v5YVE9Q,implied_volatility_mean_90,FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
3,6831678,20000000,1037,1414,0.1440,0.0695,0.5276,0.000965,0.33,0.47,dn9wMqx,implied_volatility_mean_60,FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
4,6518638,20000000,1058,1412,0.1441,0.0663,0.4429,0.000920,0.33,0.49,v5YV7lr,implied_volatility_call_150,FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,282901,20000000,1198,1106,0.5592,0.0029,0.2324,0.000010,0.00,0.05,RN5LOqz,implied_volatility_mean_skew_120,PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING
60,-660893,20000000,1188,1116,0.5793,-0.0067,0.3095,-0.000023,-0.01,-0.13,VXMor0b,implied_volatility_mean_skew_60,PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING
61,-534988,20000000,1196,1108,0.5672,-0.0054,0.2440,-0.000019,-0.01,-0.11,QnWdxYg,implied_volatility_mean_skew_90,PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING
62,-1972513,20000000,1158,1146,0.5873,-0.0201,0.3872,-0.000068,-0.05,-0.25,bNx3mqR,implied_volatility_mean_skew_20,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING


### Selecting top 5 performing datafields in the given dataset on the basis of performance.

In [11]:
a = result_st1['expression'].tolist()
top_5  = a[:5]

## BUILDING A FRAMEWORK

In [12]:
# Using different operators on the selected datafields to find the best performing combination.
opt_list = ['ts_rank(' , 'ts_quantile(' , 'ts_zscore(' , 'ts_mean(', 'ts_delay(','ts_median(' , 'ts_av_diff(' , 'ts_std_dev(']

In [13]:
avg_sharpe0 = []
for y in opt_list:
    exp_list = [f'{y}  {x}  ,360)' for x  in top_5]
    alph_list = [brain.generate_alpha(x, region= "USA",truncation = 0.08, universe = "TOP3000",pasteurization = 'ON', decay = 4 ,neutralization = "SUBINDUSTRY") for x in exp_list]
    result = brain.simulate_alpha_list_multi(s, alph_list)
    result_st = hf.prettify_result(result, detailed_tests_view=False)
    avg_sharpe = result_st['sharpe'].mean()
    avg_sharpe0.append(avg_sharpe)
    print(f'Average Sharpe for {y}) is: {avg_sharpe}')
    display(result_st)

Warning, list of alphas too short, single concurrent simulations will be used instead of multisimulations


100%|█████████████████████████████████████████████| 5/5 [01:47<00:00, 21.59s/it]


Average Sharpe for ts_rank() is: 1.1380000000000001


,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,5198578,20000000,1238,1231,0.2526,0.0529,0.0579,0.000419,0.57,1.24,gmrz2j0,"ts_rank( implied_volatility_call_180 ,360)",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
1,5237430,20000000,1240,1230,0.2527,0.0532,0.0595,0.000421,0.57,1.25,nn6bEez,"ts_rank( implied_volatility_call_150 ,360)",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
2,5146570,20000000,1236,1234,0.2563,0.0523,0.0571,0.000408,0.56,1.24,QnWd83W,"ts_rank( implied_volatility_call_270 ,360)",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
3,4283314,20000000,1238,1213,0.2449,0.0435,0.0743,0.000356,0.43,1.01,AnLdAel,"ts_rank( implied_volatility_mean_60 ,360)",PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING
4,3997443,20000000,1237,1213,0.2282,0.0406,0.0701,0.000356,0.40,0.95,kn85AVl,"ts_rank( implied_volatility_mean_90 ,360)",PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING


Warning, list of alphas too short, single concurrent simulations will be used instead of multisimulations


...

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,6178048,20000000,1207,1243,0.2076,0.0628,0.2359,0.000605,0.25,0.45,mqKOL9p,"ts_av_diff( implied_volatility_mean_90 ,360)",FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
1,7317486,20000000,1218,1252,0.3979,0.0744,0.2738,0.000374,0.24,0.55,xelMA0w,"ts_av_diff( implied_volatility_call_180 ,360)",FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
2,6964788,20000000,1218,1252,0.4038,0.0708,0.2752,0.000351,0.22,0.53,z5Aaq1K,"ts_av_diff( implied_volatility_call_270 ,360)",FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
3,6904200,20000000,1216,1254,0.3975,0.0702,0.2705,0.000353,0.22,0.53,KnVeP68,"ts_av_diff( implied_volatility_call_150 ,360)",FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
4,4242905,20000000,1205,1246,0.2159,0.0431,0.3866,0.000400,0.11,0.25,XkNqgLb,"ts_av_diff( implied_volatility_mean_60 ,360)",FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING


Warning, list of alphas too short, single concurrent simulations will be used instead of multisimulations


100%|█████████████████████████████████████████████| 5/5 [03:28<00:00, 41.80s/it]


Average Sharpe for ts_std_dev() is: 0.03800000000000001


,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,6673851,20000000,954,1729,0.0145,0.0679,0.7362,0.009339,0.21,0.29,NnKMxzp,"ts_std_dev( implied_volatility_mean_60 ,360)",FAIL,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING
1,4933363,20000000,948,1735,0.0147,0.0502,0.5932,0.006839,0.18,0.28,kn8bEwl,"ts_std_dev( implied_volatility_mean_90 ,360)",FAIL,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING
2,-1473459,20000000,939,1747,0.0144,-0.0150,0.3446,-0.002081,-0.04,-0.12,XkNqGnm,"ts_std_dev( implied_volatility_call_150 ,360)",FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
3,-1584395,20000000,936,1750,0.0145,-0.0161,0.3446,-0.002225,-0.05,-0.13,onP28pm,"ts_std_dev( implied_volatility_call_270 ,360)",FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
4,-1521956,20000000,937,1749,0.0144,-0.0155,0.3446,-0.002143,-0.05,-0.13,3EG3QmP,"ts_std_dev( implied_volatility_call_180 ,360)",FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING


In [14]:
for opt in range(len(opt_list)):
    print(f'Average Sharpe for {opt_list[opt]}) is: {avg_sharpe0[opt]:.2f}')

    

Average Sharpe for ts_rank() is: 1.14
Average Sharpe for ts_quantile() is: 1.11
Average Sharpe for ts_zscore() is: 1.00
Average Sharpe for ts_mean() is: -0.05
Average Sharpe for ts_delay() is: 0.09
Average Sharpe for ts_median() is: 0.25
Average Sharpe for ts_av_diff() is: 0.46
Average Sharpe for ts_std_dev() is: 0.04


### The combination of the operator and field that demonstrates the highest efficacy is "ts_rank." 

In [15]:
expression_list2 =[f'ts_rank({x},360)' for x  in top_5]

In [16]:
alpha_list2 = [brain.generate_alpha(x, region= "USA",truncation = 0.08, universe = "TOP3000",pasteurization = 'ON', decay = 4 ,neutralization = "SUBINDUSTRY") for x in expression_list2]

In [17]:
result2 = brain.simulate_alpha_list_multi(s, alpha_list2)

Warning, list of alphas too short, single concurrent simulations will be used instead of multisimulations


100%|█████████████████████████████████████████████| 5/5 [01:08<00:00, 13.71s/it]


In [18]:
result_st2 = hf.prettify_result(result2, detailed_tests_view=False)
result_st2

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,5198578,20000000,1238,1231,0.2526,0.0529,0.0579,0.000419,0.57,1.24,gmrz2j0,"ts_rank(implied_volatility_call_180,360)",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
1,5237430,20000000,1240,1230,0.2527,0.0532,0.0595,0.000421,0.57,1.25,nn6bEez,"ts_rank(implied_volatility_call_150,360)",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
2,5146570,20000000,1236,1234,0.2563,0.0523,0.0571,0.000408,0.56,1.24,QnWd83W,"ts_rank(implied_volatility_call_270,360)",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
3,4283314,20000000,1238,1213,0.2449,0.0435,0.0743,0.000356,0.43,1.01,AnLdAel,"ts_rank(implied_volatility_mean_60,360)",PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING
4,3997443,20000000,1237,1213,0.2282,0.0406,0.0701,0.000356,0.40,0.95,kn85AVl,"ts_rank(implied_volatility_mean_90,360)",PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING


### Utilizing the zscore() operator post-normalization via ts_rank() facilitates data set rescaling to its original scale, ensuring standardization while preserving the initial data distribution.

In [19]:
expression_list3 =[f' zscore(ts_rank({x}, 360)) ' for x  in top_5]

In [20]:
alpha_list3 = [brain.generate_alpha(x, region= "USA",truncation = 0.08, universe = "TOP3000",pasteurization = 'ON', decay = 4 ,neutralization = "SUBINDUSTRY") for x in expression_list3]

In [21]:
result3 = brain.simulate_alpha_list_multi(s, alpha_list3)

Warning, list of alphas too short, single concurrent simulations will be used instead of multisimulations


100%|█████████████████████████████████████████████| 5/5 [01:11<00:00, 14.23s/it]


In [22]:
result_st3 = hf.prettify_result(result3, detailed_tests_view=False)
result_st3

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,5249593,20000000,1238,1231,0.2523,0.0534,0.0578,0.000423,0.58,1.25,1oGvEKz,"zscore(ts_rank(implied_volatility_call_180, 3...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
1,5294127,20000000,1240,1229,0.2523,0.0538,0.0593,0.000427,0.58,1.26,nn60b93,"zscore(ts_rank(implied_volatility_call_150, 3...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
2,5197910,20000000,1236,1233,0.2560,0.0528,0.0567,0.000413,0.57,1.25,RN5XWPe,"zscore(ts_rank(implied_volatility_call_270, 3...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
3,4290291,20000000,1238,1213,0.2449,0.0436,0.0738,0.000356,0.43,1.01,XkNqEjm,"zscore(ts_rank(implied_volatility_mean_60, 36...",PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING
4,4014847,20000000,1238,1213,0.2281,0.0408,0.0697,0.000358,0.40,0.95,v5Y0gPr,"zscore(ts_rank(implied_volatility_mean_90, 36...",PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING


### Opting for the top 3 expressions due to their better performance.

In [23]:
result_st3 = result_st3[0:3]
result_st3

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,5249593,20000000,1238,1231,0.2523,0.0534,0.0578,0.000423,0.58,1.25,1oGvEKz,"zscore(ts_rank(implied_volatility_call_180, 3...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
1,5294127,20000000,1240,1229,0.2523,0.0538,0.0593,0.000427,0.58,1.26,nn60b93,"zscore(ts_rank(implied_volatility_call_150, 3...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
2,5197910,20000000,1236,1233,0.2560,0.0528,0.0567,0.000413,0.57,1.25,RN5XWPe,"zscore(ts_rank(implied_volatility_call_270, 3...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING


## DISPERSION OF CALL OPTION

### To capture a comprehensive and nuanced view of market dynamics by combining medium-term volatility expectations with short-term price sensitivity signals, we utilised the option4 dataset which includes all datafields related to dispersion of call option.

In [24]:
selected_dataset =  hf.get_datafields(s, dataset_id='option4')
selected_dataset

,id,description,dataset,category,subcategory,region,delay,universe,type,coverage,turnover,userCount,alphaCount
0,opt4_122_call_dis_delta20,The dispersion of a theoretical call option wi...,"{'id': 'option4', 'name': 'Implied Volatility ...",option,option-option,USA,1,TOP3000,MATRIX,0.8362,None,46,414
1,opt4_122_call_dis_delta25,The dispersion of a theoretical call option wi...,"{'id': 'option4', 'name': 'Implied Volatility ...",option,option-option,USA,1,TOP3000,MATRIX,0.8362,None,36,122
2,opt4_122_call_dis_delta30,The dispersion of a theoretical call option wi...,"{'id': 'option4', 'name': 'Implied Volatility ...",option,option-option,USA,1,TOP3000,MATRIX,0.8362,None,15,52
3,opt4_122_call_dis_delta35,The dispersion of a theoretical call option wi...,"{'id': 'option4', 'name': 'Implied Volatility ...",option,option-option,USA,1,TOP3000,MATRIX,0.8362,None,14,88
4,opt4_122_call_dis_delta40,The dispersion of a theoretical call option wi...,"{'id': 'option4', 'name': 'Implied Volatility ...",option,option-option,USA,1,TOP3000,MATRIX,0.8362,None,50,340
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,opt4_vola_365d,"Realized volatility, calculated over date rang...","{'id': 'option4', 'name': 'Implied Volatility ...",option,option-option,USA,1,TOP3000,MATRIX,0.8397,None,37,134
1192,opt4_vola_547d,"Realized volatility, calculated over date rang...","{'id': 'option4', 'name': 'Implied Volatility ...",option,option-option,USA,1,TOP3000,MATRIX,0.8397,None,49,334
1193,opt4_vola_60d,"Realized volatility, calculated over date rang...","{'id': 'option4', 'name': 'Implied Volatility ...",option,option-option,USA,1,TOP3000,MATRIX,0.8396,None,17,50
1194,opt4_vola_730d,"Realized volatility, calculated over date rang...","{'id': 'option4', 'name': 'Implied Volatility ...",option,option-option,USA,1,TOP3000,MATRIX,0.8397,None,29,108


### Choosing the data fields with dispersion of call options and the alpha count greater than 50 in order to minimize the search space.

In [25]:
selected_datafields = selected_dataset.query("""
        id.str.contains('call_dis', case=False) == 1 &\
        50 <= alphaCount 
    """, engine='python')

selected_datafields = selected_datafields['id'].tolist()
print("The datafields with dispersion of call options having alpha count greater than 50 are : ")
display(selected_datafields)

The datafields with dispersion of call options having alpha count greater than 50 are : 


['opt4_122_call_dis_delta20',
 'opt4_122_call_dis_delta25',
 'opt4_122_call_dis_delta30',
 'opt4_122_call_dis_delta35',
 'opt4_122_call_dis_delta40',
 'opt4_122_call_dis_delta45',
 'opt4_122_call_dis_delta50',
 'opt4_122_call_dis_delta55',
 'opt4_122_call_dis_delta60',
 'opt4_122_call_dis_delta70',
 'opt4_122_call_dis_delta80',
 'opt4_152_call_dis_delta35',
 'opt4_152_call_dis_delta40',
 'opt4_152_call_dis_delta45',
 'opt4_152_call_dis_delta60',
 'opt4_152_call_dis_delta65',
 'opt4_152_call_dis_delta75',
 'opt4_182_call_dis_delta25',
 'opt4_182_call_dis_delta30',
 'opt4_182_call_dis_delta50',
 'opt4_182_call_dis_delta55',
 'opt4_182_call_dis_delta60',
 'opt4_182_call_dis_delta75',
 'opt4_182_call_dis_delta80',
 'opt4_273_call_dis_delta80',
 'opt4_30_call_dis_delta40',
 'opt4_30_call_dis_delta50',
 'opt4_30_call_dis_delta60',
 'opt4_30_call_dis_delta80',
 'opt4_365_call_dis_delta45',
 'opt4_365_call_dis_delta60',
 'opt4_365_call_dis_delta75',
 'opt4_547_call_dis_delta35',
 'opt4_547_cal

In [26]:
print("Expressions used: ")
expression_list4 = result_st3['expression']
expression_list4

Expressions used: 


0     zscore(ts_rank(implied_volatility_call_180, 3...
1     zscore(ts_rank(implied_volatility_call_150, 3...
2     zscore(ts_rank(implied_volatility_call_270, 3...
Name: expression, dtype: object

In [27]:
final_exp_list = []
avg_fitness2 = []
avg_sharpe1 = []
for p in expression_list4:
    exp_list = [f' {x} + {p} ' for x  in selected_datafields]
    alph_list = [brain.generate_alpha(x, region= "USA",truncation = 0.08, universe = "TOP3000",pasteurization = 'ON', decay = 4 ,neutralization = "SUBINDUSTRY") for x in exp_list]
    result = brain.simulate_alpha_list_multi(s, alph_list)
    result_st = hf.prettify_result(result, detailed_tests_view=False)
    final_exp_list.append(result_st)
    display(result_st)

100%|███████████████████████████████████████████| 17/17 [06:04<00:00, 21.46s/it]


,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,5685936,20000000,1184,1181,0.2576,0.0578,0.0577,0.000449,0.63,1.32,AnL66Gl,opt4_30_call_dis_delta40 + zscore(ts_rank(im...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
1,5668225,20000000,1187,1180,0.2552,0.0576,0.0564,0.000452,0.63,1.32,99GoQG1,opt4_122_call_dis_delta50 + zscore(ts_rank(i...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
2,5652028,20000000,1187,1180,0.2552,0.0575,0.0555,0.000450,0.63,1.32,Eqm33w1,opt4_182_call_dis_delta50 + zscore(ts_rank(i...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
3,5625010,20000000,1188,1179,0.2551,0.0572,0.0553,0.000448,0.63,1.32,AnL6qKE,opt4_547_call_dis_delta65 + zscore(ts_rank(i...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
4,5636093,20000000,1187,1180,0.2552,0.0573,0.0554,0.000449,0.63,1.32,99GoQG9,opt4_122_call_dis_delta60 + zscore(ts_rank(i...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
5,5648901,20000000,1187,1180,0.2552,0.0574,0.0561,0.000450,0.63,1.32,Ln8ex86,opt4_122_call_dis_delta55 + zscore(ts_rank(i...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
6,5686179,20000000,1186,1180,0.2572,0.0578,0.0567,0.000449,0.63,1.33,Ln8eeGM,opt4_30_call_dis_delta50 + zscore(ts_rank(im...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
7,5708969,20000000,1187,1180,0.2554,0.0580,0.0567,0.000455,0.63,1.33,w5N6XN1,opt4_91_call_dis_delta50 + zscore(ts_rank(im...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
8,5688644,20000000,1187,1180,0.2554,0.0578,0.0561,0.000453,0.63,1.33,jnVLNV9,opt4_91_call_dis_delta55 + zscore(ts_rank(im...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
9,5614592,20000000,1187,1178,0.2579,0.0571,0.0556,0.000443,0.62,1.32,NnKMkKo,opt4_730_call_dis_delta80 + zscore(ts_rank(i...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING


100%|███████████████████████████████████████████| 17/17 [05:47<00:00, 20.45s/it]


,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,5771793,20000000,1186,1180,0.2577,0.0587,0.0595,0.000455,0.64,1.34,w5N6OmQ,opt4_30_call_dis_delta40 + zscore(ts_rank(im...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
1,5753728,20000000,1188,1179,0.2552,0.0585,0.0594,0.000459,0.64,1.33,nn60k7E,opt4_152_call_dis_delta40 + zscore(ts_rank(i...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
2,5789463,20000000,1188,1179,0.2555,0.0589,0.0586,0.000461,0.64,1.34,aNRJlM6,opt4_91_call_dis_delta50 + zscore(ts_rank(im...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
3,5759058,20000000,1186,1180,0.2560,0.0586,0.0603,0.000457,0.64,1.33,ZjXZqNY,opt4_60_call_dis_delta40 + zscore(ts_rank(im...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
4,5708989,20000000,1190,1177,0.2553,0.0580,0.0571,0.000455,0.64,1.34,QnWeq0X,opt4_547_call_dis_delta65 + zscore(ts_rank(i...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
5,5699407,20000000,1188,1177,0.2578,0.0579,0.0577,0.000449,0.64,1.34,w5N6O51,opt4_30_call_dis_delta60 + zscore(ts_rank(im...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
6,5773288,20000000,1187,1178,0.2574,0.0587,0.0584,0.000456,0.64,1.35,2vGEQ06,opt4_30_call_dis_delta50 + zscore(ts_rank(im...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
7,5740349,20000000,1189,1177,0.2577,0.0584,0.0577,0.000453,0.64,1.34,Jn3lYwO,opt4_273_call_dis_delta80 + zscore(ts_rank(i...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
8,5732811,20000000,1189,1178,0.2553,0.0583,0.0573,0.000457,0.64,1.33,QnWer8K,opt4_182_call_dis_delta50 + zscore(ts_rank(i...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
9,5733704,20000000,1188,1179,0.2552,0.0583,0.0590,0.000457,0.64,1.33,bNxp2Rl,opt4_152_call_dis_delta45 + zscore(ts_rank(i...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING


100%|███████████████████████████████████████████| 17/17 [05:57<00:00, 21.04s/it]


,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,5606962,20000000,1182,1183,0.2613,0.0570,0.0572,0.000436,0.62,1.32,pnW0zao,opt4_30_call_dis_delta40 + zscore(ts_rank(im...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
1,5587032,20000000,1184,1182,0.2609,0.0568,0.0561,0.000435,0.62,1.32,bNxpPaN,opt4_30_call_dis_delta50 + zscore(ts_rank(im...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
2,5636666,20000000,1184,1182,0.2591,0.0573,0.0560,0.000442,0.62,1.32,onP232J,opt4_91_call_dis_delta50 + zscore(ts_rank(im...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
3,5548718,20000000,1186,1181,0.2588,0.0564,0.0547,0.000436,0.62,1.32,z5AaJjG,opt4_547_call_dis_delta65 + zscore(ts_rank(i...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
4,5613552,20000000,1185,1182,0.2591,0.0571,0.0555,0.000441,0.62,1.32,xelMPMp,opt4_91_call_dis_delta55 + zscore(ts_rank(im...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
5,5548520,20000000,1185,1182,0.2589,0.0564,0.0547,0.000436,0.61,1.31,ZjXZxNn,opt4_122_call_dis_delta60 + zscore(ts_rank(i...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
6,5510372,20000000,1186,1181,0.2589,0.0560,0.0548,0.000433,0.61,1.31,xelMm1p,opt4_730_call_dis_delta65 + zscore(ts_rank(i...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
7,5573073,20000000,1185,1182,0.2589,0.0567,0.0554,0.000438,0.61,1.31,On6ekgb,opt4_122_call_dis_delta55 + zscore(ts_rank(i...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
8,5605240,20000000,1183,1183,0.2587,0.0570,0.0577,0.000441,0.61,1.30,5LGR9YJ,opt4_122_call_dis_delta35 + zscore(ts_rank(i...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
9,5514978,20000000,1185,1182,0.2588,0.0561,0.0553,0.000433,0.61,1.30,aNRJWXO,opt4_365_call_dis_delta45 + zscore(ts_rank(i...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING


In [28]:
avg_fitness2 = []
for o in final_exp_list: 
    avg_fitness_ = o['fitness'].mean()
    avg_fitness2.append(avg_fitness_)
    
avg_sharpe1 = []
for o in final_exp_list: 
    avg_sharpe = o['sharpe'].mean()
    avg_sharpe1.append(avg_sharpe)

for exp in range(len(expression_list4)):
    print(f'Average Sharpe and Fitness for {expression_list4[exp]} is: {avg_sharpe1[exp]:.2f} , {avg_fitness2[exp]:.2f}')

    

Average Sharpe and Fitness for  zscore(ts_rank(implied_volatility_call_180, 360))  is: 1.31 , 0.62
Average Sharpe and Fitness for  zscore(ts_rank(implied_volatility_call_150, 360))  is: 1.32 , 0.63
Average Sharpe and Fitness for  zscore(ts_rank(implied_volatility_call_270, 360))  is: 1.30 , 0.61


### Since the average performance metrics from the second expression turned out to be best, we'll continue with it. 

In [29]:
result_st = final_exp_list[1]

In [30]:
# Selecting alphas having best fitness among all others.
selected_expressions = result_st[result_st['fitness'] == max(result_st['fitness'])]['expression']
print("Alpha Expressions with highest fitness are: ")
selected_expressions

Alpha Expressions with highest fitness are: 


0      opt4_30_call_dis_delta40 +  zscore(ts_rank(im...
1      opt4_152_call_dis_delta40 +  zscore(ts_rank(i...
2      opt4_91_call_dis_delta50 +  zscore(ts_rank(im...
3      opt4_60_call_dis_delta40 +  zscore(ts_rank(im...
4      opt4_547_call_dis_delta65 +  zscore(ts_rank(i...
5      opt4_30_call_dis_delta60 +  zscore(ts_rank(im...
6      opt4_30_call_dis_delta50 +  zscore(ts_rank(im...
7      opt4_273_call_dis_delta80 +  zscore(ts_rank(i...
8      opt4_182_call_dis_delta50 +  zscore(ts_rank(i...
9      opt4_152_call_dis_delta45 +  zscore(ts_rank(i...
10     opt4_91_call_dis_delta55 +  zscore(ts_rank(im...
11     opt4_122_call_dis_delta55 +  zscore(ts_rank(i...
12     opt4_122_call_dis_delta50 +  zscore(ts_rank(i...
13     opt4_122_call_dis_delta45 +  zscore(ts_rank(i...
Name: expression, dtype: object

In [31]:
# Using different operators on the selected alphas to find the best performing combination.
opt_list = ['ts_rank(' , 'ts_quantile(' , 'ts_mean(', 'ts_delay(','ts_median(' , 'ts_av_diff(' , 'ts_std_dev(']

In [32]:
final_exp_list1 = []
for opt in opt_list:
    exp_list = [f" {opt} { x.split('+')[0] } ,30) + {x.split('+')[1]} " for x  in selected_expressions]
    alph_list = [brain.generate_alpha(x, region= "USA",truncation = 0.08, universe = "TOP3000",pasteurization = 'ON', decay = 4 ,neutralization = "SUBINDUSTRY") for x in exp_list]
    result = brain.simulate_alpha_list_multi(s, alph_list)
    result_st = hf.prettify_result(result, detailed_tests_view=False)
    final_exp_list1.append(result_st)
    display(result_st)

100%|█████████████████████████████████████████████| 5/5 [02:00<00:00, 24.20s/it]


,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,6006628,20000000,1186,1179,0.2909,0.0611,0.0527,0.000420,0.66,1.45,2vGQWaw,"ts_rank( opt4_30_call_dis_delta40 ,30) + ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
1,5703620,20000000,1186,1181,0.2903,0.0580,0.0563,0.000399,0.62,1.38,bNxVnrr,"ts_rank( opt4_60_call_dis_delta40 ,30) + ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
2,5692184,20000000,1186,1179,0.2950,0.0579,0.0578,0.000392,0.61,1.37,On6qVbd,"ts_rank( opt4_30_call_dis_delta50 ,30) + ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
3,5710173,20000000,1185,1182,0.2982,0.0581,0.0545,0.000389,0.61,1.38,8OG7Yao,"ts_rank( opt4_91_call_dis_delta50 ,30) + ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
4,5715390,20000000,1186,1181,0.3087,0.0581,0.0495,0.000376,0.60,1.38,AnLYmOe,"ts_rank( opt4_547_call_dis_delta65 ,30) + ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
5,5606310,20000000,1185,1182,0.2996,0.0570,0.0529,0.000381,0.59,1.36,rb0EVJ1,"ts_rank( opt4_91_call_dis_delta55 ,30) + ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
6,5639535,20000000,1185,1182,0.3024,0.0573,0.0505,0.000379,0.59,1.36,mqKJ10W,"ts_rank( opt4_122_call_dis_delta55 ,30) + ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
7,5660779,20000000,1184,1183,0.3021,0.0576,0.0529,0.000381,0.59,1.36,6RG3JJK,"ts_rank( opt4_122_call_dis_delta50 ,30) + ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
8,5601609,20000000,1183,1184,0.3002,0.0570,0.0572,0.000379,0.59,1.35,0AG966G,"ts_rank( opt4_122_call_dis_delta45 ,30) + ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
9,5547197,20000000,1187,1179,0.3017,0.0564,0.0556,0.000374,0.58,1.34,en1gvdg,"ts_rank( opt4_30_call_dis_delta60 ,30) + ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING


100%|█████████████████████████████████████████████| 5/5 [01:55<00:00, 23.03s/it]


,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,5821957,20000000,1186,1179,0.3931,0.0592,0.0414,0.000301,0.61,1.58,kn8b9mg,"ts_quantile( opt4_30_call_dis_delta40 ,30) ...",PASS,PASS,FAIL,FAIL,PASS,PASS,PASS,WARNING
1,5133505,20000000,1188,1178,0.3874,0.0522,0.0516,0.000269,0.52,1.43,Ln8q3z1,"ts_quantile( opt4_60_call_dis_delta40 ,30) ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
2,5193003,20000000,1182,1184,0.4042,0.0528,0.0438,0.000261,0.51,1.41,jnVWOdo,"ts_quantile( opt4_91_call_dis_delta50 ,30) ...",PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING
3,5301862,20000000,1183,1183,0.4490,0.0539,0.0357,0.000240,0.50,1.43,kn8b916,"ts_quantile( opt4_547_call_dis_delta65 ,30)...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
4,5049133,20000000,1185,1179,0.4013,0.0513,0.0534,0.000256,0.49,1.36,bNxVAPr,"ts_quantile( opt4_30_call_dis_delta50 ,30) ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
5,5104417,20000000,1182,1184,0.4147,0.0519,0.0459,0.000250,0.49,1.38,KnVqZ5E,"ts_quantile( opt4_122_call_dis_delta50 ,30)...",PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING
6,4978630,20000000,1183,1183,0.4094,0.0506,0.0423,0.000247,0.48,1.36,XkNPMYb,"ts_quantile( opt4_91_call_dis_delta55 ,30) ...",PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING
7,5079481,20000000,1183,1183,0.4184,0.0516,0.0440,0.000247,0.48,1.37,VXMqezM,"ts_quantile( opt4_122_call_dis_delta55 ,30)...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
8,4937263,20000000,1182,1184,0.4102,0.0502,0.0515,0.000245,0.47,1.34,en1gEV6,"ts_quantile( opt4_122_call_dis_delta45 ,30)...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
9,4864373,20000000,1185,1179,0.4207,0.0495,0.0479,0.000235,0.45,1.31,AnLYEOX,"ts_quantile( opt4_30_call_dis_delta60 ,30) ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING


100%|█████████████████████████████████████████████| 5/5 [01:59<00:00, 23.97s/it]


...

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,5716813,20000000,1191,1179,0.2540,0.0581,0.0570,0.000458,0.64,1.33,Mbgqvvz,"ts_median( opt4_91_call_dis_delta50 ,30) + ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
1,5704106,20000000,1191,1179,0.2540,0.0580,0.0561,0.000457,0.64,1.33,QnWqRxQ,"ts_median( opt4_30_call_dis_delta50 ,30) + ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
2,5707132,20000000,1191,1179,0.2540,0.0580,0.0571,0.000457,0.64,1.33,8OG7W0l,"ts_median( opt4_182_call_dis_delta50 ,30) +...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
3,5707318,20000000,1192,1178,0.2540,0.0580,0.0568,0.000457,0.64,1.34,AnLYXpd,"ts_median( opt4_547_call_dis_delta65 ,30) +...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
4,5718299,20000000,1191,1179,0.2539,0.0581,0.0570,0.000458,0.64,1.33,8OG70rz,"ts_median( opt4_122_call_dis_delta50 ,30) +...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
5,5728156,20000000,1191,1180,0.2539,0.0582,0.0569,0.000459,0.64,1.33,6RG3W1p,"ts_median( opt4_122_call_dis_delta45 ,30) +...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
6,5712042,20000000,1191,1180,0.2539,0.0581,0.0570,0.000457,0.64,1.33,w5NO77d,"ts_median( opt4_152_call_dis_delta45 ,30) +...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
7,5701132,20000000,1191,1179,0.2539,0.0580,0.0571,0.000456,0.64,1.33,WNYnrrO,"ts_median( opt4_91_call_dis_delta55 ,30) + ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
8,5712453,20000000,1191,1179,0.2539,0.0581,0.0572,0.000457,0.64,1.33,EqmYjrK,"ts_median( opt4_122_call_dis_delta55 ,30) +...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
9,5696047,20000000,1190,1180,0.2540,0.0579,0.0574,0.000456,0.63,1.32,RN5qxOa,"ts_median( opt4_30_call_dis_delta40 ,30) + ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING


100%|█████████████████████████████████████████████| 5/5 [01:48<00:00, 21.80s/it]


,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,5693043,20000000,1189,1176,0.2575,0.0579,0.0572,0.000450,0.64,1.35,en1gYGp,"ts_av_diff( opt4_30_call_dis_delta50 ,30) +...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
1,5706187,20000000,1189,1177,0.2579,0.0580,0.0564,0.000450,0.64,1.36,gmrwlgl,"ts_av_diff( opt4_30_call_dis_delta40 ,30) +...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
2,5609389,20000000,1190,1177,0.2562,0.0570,0.0576,0.000445,0.63,1.33,RN5qgqd,"ts_av_diff( opt4_60_call_dis_delta40 ,30) +...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
3,5592499,20000000,1190,1176,0.2554,0.0569,0.0561,0.000445,0.63,1.33,jnVWpG9,"ts_av_diff( opt4_547_call_dis_delta65 ,30) ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
4,5646032,20000000,1189,1176,0.2579,0.0574,0.0566,0.000445,0.63,1.34,rb0EqnJ,"ts_av_diff( opt4_30_call_dis_delta60 ,30) +...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
5,5630410,20000000,1190,1176,0.2580,0.0572,0.0556,0.000444,0.63,1.34,bNxVwVZ,"ts_av_diff( opt4_273_call_dis_delta80 ,30) ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
6,5604076,20000000,1190,1177,0.2555,0.0570,0.0564,0.000446,0.63,1.33,kn8bwGk,"ts_av_diff( opt4_182_call_dis_delta50 ,30) ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
7,5616425,20000000,1190,1177,0.2554,0.0571,0.0578,0.000447,0.63,1.34,NnKqdzo,"ts_av_diff( opt4_152_call_dis_delta40 ,30) ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
8,5631454,20000000,1190,1177,0.2557,0.0573,0.0572,0.000448,0.63,1.34,RN5qOv0,"ts_av_diff( opt4_91_call_dis_delta50 ,30) +...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
9,5596364,20000000,1190,1177,0.2554,0.0569,0.0577,0.000445,0.63,1.33,WNYnZzj,"ts_av_diff( opt4_152_call_dis_delta45 ,30) ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING


100%|█████████████████████████████████████████████| 5/5 [02:34<00:00, 31.00s/it]


,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,5692914,20000000,1190,1179,0.2542,0.0579,0.0579,0.000455,0.63,1.32,w5NOrG2,"ts_std_dev( opt4_91_call_dis_delta50 ,30) +...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
1,5629859,20000000,1191,1179,0.2542,0.0572,0.0573,0.000450,0.63,1.32,ZjXqJzx,"ts_std_dev( opt4_547_call_dis_delta65 ,30) ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
2,5677455,20000000,1190,1179,0.2542,0.0577,0.0578,0.000454,0.63,1.32,2vGQY6x,"ts_std_dev( opt4_30_call_dis_delta60 ,30) +...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
3,5678389,20000000,1189,1180,0.2542,0.0577,0.0582,0.000454,0.63,1.32,RN5qgzn,"ts_std_dev( opt4_30_call_dis_delta50 ,30) +...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
4,5656169,20000000,1191,1179,0.2542,0.0575,0.0572,0.000452,0.63,1.32,rb0ExGE,"ts_std_dev( opt4_182_call_dis_delta50 ,30) ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
5,5666015,20000000,1191,1179,0.2542,0.0576,0.0576,0.000453,0.63,1.32,1oG1MlM,"ts_std_dev( opt4_152_call_dis_delta45 ,30) ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
6,5662144,20000000,1190,1179,0.2542,0.0576,0.0575,0.000453,0.63,1.32,en1gjGz,"ts_std_dev( opt4_91_call_dis_delta55 ,30) +...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
7,5667407,20000000,1191,1179,0.2542,0.0576,0.0575,0.000453,0.63,1.32,VXMqA3J,"ts_std_dev( opt4_122_call_dis_delta55 ,30) ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
8,5674979,20000000,1191,1179,0.2542,0.0577,0.0575,0.000454,0.63,1.32,5LG3Pwn,"ts_std_dev( opt4_122_call_dis_delta50 ,30) ...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
9,5675389,20000000,1189,1180,0.2541,0.0577,0.0599,0.000454,0.62,1.31,AnLYAzR,"ts_std_dev( opt4_30_call_dis_delta40 ,30) +...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING


In [33]:
avg_sharpe3 = []
for o in final_exp_list1: 
    avg_sharpe = o['sharpe'].mean()
    avg_sharpe3.append(avg_sharpe)

for opt in range(len(opt_list)):
    print(f'Average Sharpe for {opt_list[opt]}) is: {avg_sharpe3[opt]:.2f}')
    

Average Sharpe for ts_rank() is: 1.36
Average Sharpe for ts_quantile() is: 1.36
Average Sharpe for ts_mean() is: 1.33
Average Sharpe for ts_delay() is: 1.32
Average Sharpe for ts_median() is: 1.33
Average Sharpe for ts_av_diff() is: 1.34
Average Sharpe for ts_std_dev() is: 1.32


In [34]:
# Using Fitness as the tie-breaker

avg_fitness3 = []
for o in final_exp_list1: 
    avg_fitness_ = o['fitness'].mean()
    avg_fitness3.append(avg_fitness_)
    
for opt in range(2):    
    print(f"Average Fitness for {opt_list[opt]}) is: {avg_fitness3[opt]:.2f}")
    

Average Fitness for ts_rank() is: 0.59
Average Fitness for ts_quantile() is: 0.48


### Clearly ts_rank() demonstrates the highest efficacy, so we take this particular expression to further analyse.

In [35]:
selected_alpha = 'zscore(ts_rank(implied_volatility_call_150, 360)) + ts_rank(opt4_30_call_dis_delta40, 30)'

In [36]:
alpha_temp =[ brain.generate_alpha(selected_alpha , region= "USA",truncation = 0.08, universe = "TOP3000",pasteurization = 'ON', decay = 4 ,neutralization = "SUBINDUSTRY") ]
result_temp = brain.simulate_alpha_list_multi(s, alpha_temp)
results_temp = hf.prettify_result(result_temp, detailed_tests_view=False)

Warning, list of alphas too short, single concurrent simulations will be used instead of multisimulations


100%|█████████████████████████████████████████████| 1/1 [00:38<00:00, 38.85s/it]


In [37]:
display(results_temp)

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,6006628,20000000,1186,1179,0.2909,0.0611,0.0527,0.00042,0.66,1.45,aNRlrW6,"zscore(ts_rank(implied_volatility_call_150, 36...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING


## ANALYSING PERFORMANCE METRICS

### With turnover currently at 29%, we aim to reduce it by employing turnover-reducing operators. It's crucial to note that the alpha itself generates the signal.

In [38]:
turnover_red_opt_list = ['trade_when(rank(implied_volatility_call_150)>0.1,  ' , 'trade_when(rank(implied_volatility_call_150)>0.15,  ', 'trade_when(rank(implied_volatility_call_150)>0.2,  ' , 'trade_when(rank(implied_volatility_call_150)>0.25,  ', 'trade_when(rank(implied_volatility_call_150)>0.3,  ', 'trade_when(rank(implied_volatility_call_150)>0.35,  ',  'if_else(rank(implied_volatility_call_150)>0.1, ' , 'if_else(rank(implied_volatility_call_150)>0.15, ' , 'if_else(rank(implied_volatility_call_150)>0.2, ' , 'if_else(rank(implied_volatility_call_150)>0.25, ', 'if_else(rank(implied_volatility_call_150)>0.3, ','if_else(rank(implied_volatility_call_150)>0.35, ']

exp_list_after = [f" {opt} { selected_alpha } , -1) " for opt in turnover_red_opt_list]
alph_list_after = [brain.generate_alpha(x, region= "USA",truncation = 0.08, universe = "TOP3000",pasteurization = 'ON', decay = 4 ,neutralization = "SUBINDUSTRY") for x in exp_list_after]
result_after = brain.simulate_alpha_list_multi(s, alph_list_after)
result_st_after = hf.prettify_result(result_after, detailed_tests_view=False)
display(result_st_after)

100%|█████████████████████████████████████████████| 4/4 [02:05<00:00, 31.36s/it]


,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,4934792,20000000,1298,1260,0.2048,0.0502,0.0380,0.000490,0.73,1.47,KnV6Kp8,trade_when(rank(implied_volatility_call_150)>...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
1,4679890,20000000,1293,1244,0.1965,0.0476,0.0338,0.000484,0.71,1.44,w5N18AQ,trade_when(rank(implied_volatility_call_150)>...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
2,4425527,20000000,1284,1226,0.1885,0.0450,0.0522,0.000477,0.68,1.40,2vGR1x6,trade_when(rank(implied_volatility_call_150)>...,PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING
3,4279552,20000000,1253,1182,0.1723,0.0435,0.0802,0.000505,0.68,1.35,nn6o1qM,trade_when(rank(implied_volatility_call_150)>...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
4,4310327,20000000,1269,1202,0.1813,0.0438,0.0631,0.000483,0.67,1.37,gmrKYgg,trade_when(rank(implied_volatility_call_150)>...,PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING
5,6355673,20000000,1161,1208,0.2792,0.0646,0.0662,0.000463,0.65,1.35,RN5YJl1,if_else(rank(implied_volatility_call_150)>0.1...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
6,4074802,20000000,1230,1154,0.1646,0.0414,0.0990,0.000503,0.64,1.28,z5AjvGo,trade_when(rank(implied_volatility_call_150)>...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
7,6450971,20000000,1139,1227,0.2713,0.0656,0.0780,0.000484,0.62,1.26,5LG9ZA1,if_else(rank(implied_volatility_call_150)>0.1...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
8,6550492,20000000,1113,1248,0.2646,0.0666,0.0879,0.000503,0.59,1.18,GnQ6qzJ,if_else(rank(implied_volatility_call_150)>0.2...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
9,6619412,20000000,1081,1271,0.2574,0.0673,0.1041,0.000523,0.57,1.12,Ln86LX1,if_else(rank(implied_volatility_call_150)>0.2...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING


### Selecting the lowest turnover Alpha Expression

In [39]:
result_st_after.sort_values(by='turnover', ascending = True, inplace = False)
result_st_after

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,4934792,20000000,1298,1260,0.2048,0.0502,0.0380,0.000490,0.73,1.47,KnV6Kp8,trade_when(rank(implied_volatility_call_150)>...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
1,4679890,20000000,1293,1244,0.1965,0.0476,0.0338,0.000484,0.71,1.44,w5N18AQ,trade_when(rank(implied_volatility_call_150)>...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
2,4425527,20000000,1284,1226,0.1885,0.0450,0.0522,0.000477,0.68,1.40,2vGR1x6,trade_when(rank(implied_volatility_call_150)>...,PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING
3,4279552,20000000,1253,1182,0.1723,0.0435,0.0802,0.000505,0.68,1.35,nn6o1qM,trade_when(rank(implied_volatility_call_150)>...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
4,4310327,20000000,1269,1202,0.1813,0.0438,0.0631,0.000483,0.67,1.37,gmrKYgg,trade_when(rank(implied_volatility_call_150)>...,PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS,WARNING
5,6355673,20000000,1161,1208,0.2792,0.0646,0.0662,0.000463,0.65,1.35,RN5YJl1,if_else(rank(implied_volatility_call_150)>0.1...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
6,4074802,20000000,1230,1154,0.1646,0.0414,0.0990,0.000503,0.64,1.28,z5AjvGo,trade_when(rank(implied_volatility_call_150)>...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
7,6450971,20000000,1139,1227,0.2713,0.0656,0.0780,0.000484,0.62,1.26,5LG9ZA1,if_else(rank(implied_volatility_call_150)>0.1...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
8,6550492,20000000,1113,1248,0.2646,0.0666,0.0879,0.000503,0.59,1.18,GnQ6qzJ,if_else(rank(implied_volatility_call_150)>0.2...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING
9,6619412,20000000,1081,1271,0.2574,0.0673,0.1041,0.000523,0.57,1.12,Ln86LX1,if_else(rank(implied_volatility_call_150)>0.2...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING


In [40]:
# Selecting the alpha with least turnover
final_alpha = result_st_after[result_st_after['turnover'] == min(result_st_after['turnover'])]['expression'].tolist()[0]
final_alpha

' trade_when(rank(implied_volatility_call_150)>0.35,   zscore(ts_rank(implied_volatility_call_150, 360)) + ts_rank(opt4_30_call_dis_delta40, 30) , -1) '

In [41]:
# Selecting the best neutralization for the selected alpha
neut_list = ['MARKET' , 'SECTOR' , 'INDUSTRY' , 'SUBINDUSTRY', 'SLOW' , 'FAST' , 'SLOW_AND_FAST']

final_alphas = []
for neut in neut_list:
    alpha_final = [brain.generate_alpha(final_alpha , region= "USA",truncation = 0.08, universe = "TOP3000",pasteurization = 'ON', decay = 4 ,neutralization = neut)]
    result_final = brain.simulate_alpha_list_multi(s, alpha_final)
    results_final = hf.prettify_result(result_final, detailed_tests_view=False)
    final_alphas.append(results_final)
    print(f" For {neut} neutralization, the results are as follows: ")
    display(results_final)

Warning, list of alphas too short, single concurrent simulations will be used instead of multisimulations


100%|█████████████████████████████████████████████| 1/1 [00:33<00:00, 33.09s/it]


 For MARKET neutralization, the results are as follows: 


,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,3411108,20000000,1316,1145,0.1407,0.0347,0.2288,0.000493,0.25,0.5,ZjXxPGY,trade_when(rank(implied_volatility_call_150)>...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING


Warning, list of alphas too short, single concurrent simulations will be used instead of multisimulations


...

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,3401204,20000000,1542,1559,0.1762,0.0346,0.0921,0.000392,0.52,1.17,gmrKAeM,trade_when(rank(implied_volatility_call_150)>...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING


Warning, list of alphas too short, single concurrent simulations will be used instead of multisimulations


100%|████████████████████████████████████████████| 1/1 [02:03<00:00, 123.48s/it]


 For SLOW_AND_FAST neutralization, the results are as follows: 


,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,4522348,20000000,1540,1561,0.192,0.046,0.0381,0.000479,1.06,2.16,NnK6m5q,trade_when(rank(implied_volatility_call_150)>...,PASS,PASS,PASS,PASS,PASS,PASS,PASS,WARNING


## FINAL ALPHA EXPRESSION

In [42]:
final_alpha=final_alphas[4]

In [43]:
print("Final alpha is:\n",final_alpha['expression'].tolist()[0])
display(final_alpha)

Final alpha is:
  trade_when(rank(implied_volatility_call_150)>0.35,   zscore(ts_rank(implied_volatility_call_150, 360)) + ts_rank(opt4_30_call_dis_delta40, 30) , -1) 


,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,5418448,20000000,1542,1560,0.1849,0.0551,0.0262,0.000596,1.36,2.49,ZjXx5R1,trade_when(rank(implied_volatility_call_150)>...,PASS,PASS,PASS,PASS,PASS,PASS,PASS,WARNING


### Performing the rank() test :

In [44]:
test_alpha = f"rank({final_alpha['expression'].tolist()[0]})"
print(test_alpha)

rank( trade_when(rank(implied_volatility_call_150)>0.35,   zscore(ts_rank(implied_volatility_call_150, 360)) + ts_rank(opt4_30_call_dis_delta40, 30) , -1) )


In [45]:
test_alpha_final = [brain.generate_alpha(test_alpha , region= "USA",truncation = 0.08, universe = "TOP3000",pasteurization = 'ON', decay = 4 ,neutralization = 'SLOW')]
test_result_final = brain.simulate_alpha_list_multi(s, test_alpha_final)
test_results_final = hf.prettify_result(test_result_final, detailed_tests_view=False)
display(test_results_final)

Warning, list of alphas too short, single concurrent simulations will be used instead of multisimulations


100%|████████████████████████████████████████████| 1/1 [02:03<00:00, 123.90s/it]


,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_themes
0,4938224,20000000,1490,1611,0.1861,0.0502,0.0229,0.000539,1.28,2.47,GnQ76aZ,rank( trade_when(rank(implied_volatility_call_...,PASS,PASS,PASS,PASS,PASS,PASS,PASS,WARNING


### The metrics remain consistent, indicating that our alpha model is not exhibiting signs of overfitting.

## Overfitting Prevention Measures:
### 1. In the process of refining our alpha idea, we carefully considered not only the single best-performing expression but rather examined the top three or five performing expressions.

### 2. The initiation of the alpha idea involved adhering to default settings, prioritizing the enhancement of robustness in our hypothesis over continual adjustment of performance metrics at each step.

### 3. In the final assessment, a rank() test was conducted on our ultimate alpha expression to assess and mitigate the risk of overfitting.